<center><img src="Slide1.png" width = "800"/></center>

### <b> InSAR Coherent Change Detection over Aleppo, Syria (2016)
    
<b><i> Corey Scher<sup>1</sup>
<i>Jamon Van Den Hoek<sup>2</sup>
    
<i><font size=2> 1. Department of Earth and Environmental Sciences | Graduate Center at the City University of New York
    
<i><font size=2> 2. College of Earth, Ocean, and Atmospheric Sciences | Oregon State University

### Introduction

   In this training we will walk through an example of bi-temporal coherent change detection over Aleppo, Syria to map proxies for urban damage that occurred as a result of air bombardments in 2016. We will be using the [Alaska Satellite Facility's](https://asf.alaska.edu/) [Vertex tool](https://search.asf.alaska.edu/#/) to generate two synthetic aperture radar [interferogram](https://appliedsciences.nasa.gov/sites/default/files/Session4-SAR-English_0.pdf) (InSAR) datasets to look for anomalous decreases in InSAR measurements of [coherence](https://en.wikipedia.org/wiki/Coherence_(physics) that occurred following a series of airstrike bombardments. This workflow can be completed either using the ASF [Hyp3 Python SDK](https://hyp3-docs.asf.alaska.edu/), as we will demonstrate in this notebook, or by generating InSAR coherence images manually using the ASF Vertex [user interface](https://search.asf.alaska.edu/#/) and raster band math in a geographic information system interface, such as [QGIS](https://www.qgis.org/en/site/).

In [253]:
import os
import folium
import shapely.wkt
import pandas as pd
import asf_search as asf
from hyp3_sdk import HyP3
import shapely.geometry as shp
import matplotlib.pyplot as plt

In [71]:
#import credentials from .config file
credentials = pd.read_csv('/Users/coreyscher/Documents/GitHub/arset/hyp3.config', 
                          header = None, 
                          names = ['names', 'values']).set_index('names')

In [75]:
#authenticate to Hyp3 

hyp3 = HyP3(username=credentials.loc['username'][0], 
            password=credentials.loc['password'][0])

In [327]:
wkt = "POLYGON((37.0072 36.091,37.3326 36.091,37.3326 36.3183,37.0072 36.3183,37.0072 36.091))"
results = asf.geo_search(platform=[asf.PLATFORM.SENTINEL1], 
                         intersectsWith=wkt, 
                         maxResults=10,
                        processingLevel = 'SLC',
                        end = '2015-05-05').data


In [328]:
aoi = shapely.wkt.loads("POLYGON((37.0072 36.091,37.3326 36.091,37.3326 36.3183,37.0072 36.3183,37.0072 36.091))")

In [329]:
def addFootprint(result):
    footprint = shp.Polygon(result.geometry['coordinates'][0])
    gjson = result.geojson()
    fid = gjson['properties']['fileID']
    return fid, footprint

In [330]:
gdf = gpd.GeoDataFrame(list(map(addFootprint, results)), 
                       columns = ['fileID', 'geometry'])


In [331]:
# Add the AOI to the map for easy viewing

gdf = pd.concat([gpd.GeoDataFrame([{'geometry': aoi, 'fileID': 'aoi'}]), gdf])

In [359]:
mapa = folium.Map([36.207522, 37.154076],
                zoom_start=7,
                  tiles='cartodbpositron')

gdf.iloc[0:8].explore(m = mapa, 
            column = 'fileID',
           style_kwds = {'opacity': 1, 'fill': False})

In [374]:
# The second first image in the stack looks like a good reference and is close to the date of the UNOSAT damage mapping in September 2016

reference_scene = gdf.iloc[7]['fileID']
print(reference_scene)

#We can use this ID to generate an InSAR stack with the ASF tools 

insar_stack  = asf.stack_from_id(gdf.iloc[7]['fileID'])

S1A_IW_SLC__1SDV_20150405T033416_20150405T033443_005343_006C68_60F6-SLC


In [369]:
# Let's filter the stack for dates that are of interest

picks = []

for i in insar_stack:
    start = pd.to_datetime(i.properties['startTime'])
    
    # This will be our first interferogram
    if pd.to_datetime('2014-10-01') < start < pd.to_datetime('2014-10-15'):
        picks.append(i)
    #This will be our second interferogram, corresponding with the first UNOSAT damage assessment in April-May 2015
    if pd.to_datetime('2015-04-25') < start < pd.to_datetime('2015-05-17'):
        picks.append(i)
        
    #This will be our third interferogram, corresponding with the midpoint between UNOSAT damage assessments
    if pd.to_datetime('2015-09-01') < start < pd.to_datetime('2015-09-15'):
        picks.append(i)
        
    #This will be our second interferogram, corresponding with the first UNOSAT damage assessment in April-May 2015
    if pd.to_datetime('2016-04-25') < start < pd.to_datetime('2016-05-15'):
        picks.append(i)
        
    #This will be our second interferogram, corresponding with the first UNOSAT damage assessment in April-May 2015
    if pd.to_datetime('2016-09-01') < start < pd.to_datetime('2016-09-10'):
        picks.append(i)

In [370]:
for i in picks:
    print(pd.to_datetime(i.properties['startTime']))

2014-10-07 03:34:18
2015-05-11 03:34:18
2015-09-08 03:34:19
2016-05-05 03:34:14
2016-09-02 03:34:20


In [371]:
# Lets plot these on a map and see how they look, making certain we have full coverage over Aleppo

stack = gpd.GeoDataFrame(list(map(addFootprint, picks)), 
                       columns = ['fileID', 'geometry'])

mapb = folium.Map([36.207522, 37.154076],
                zoom_start=7,
                  tiles='cartodbpositron')
stack.explore(m = mapb, column = 'fileID',
           style_kwds = {'opacity': 1, 'fill': False})

In [373]:
i.properties

{'beamModeType': 'IW',
 'browse': [],
 'bytes': '4530234448',
 'centerLat': '36.3125',
 'centerLon': '37.0103',
 'faradayRotation': None,
 'fileID': 'S1A_IW_SLC__1SDV_20160902T033420_20160902T033447_012868_01451F_0B2F-SLC',
 'fileName': 'S1A_IW_SLC__1SDV_20160902T033420_20160902T033447_012868_01451F_0B2F.zip',
 'flightDirection': 'DESCENDING',
 'frameNumber': '471',
 'granuleType': 'SENTINEL_1A_FRAME',
 'groupID': 'S1A_IWDV_0471_0476_012868_021',
 'insarStackId': None,
 'md5sum': '7938c3409303c0f9ebd6aea835e0caf3',
 'offNadirAngle': None,
 'orbit': '12868',
 'pathNumber': '21',
 'perpendicularBaseline': -53,
 'platform': 'Sentinel-1A',
 'pointingAngle': None,
 'polarization': 'VV+VH',
 'processingDate': '2016-09-02T03:34:20.000000',
 'processingLevel': 'SLC',
 'sceneName': 'S1A_IW_SLC__1SDV_20160902T033420_20160902T033447_012868_01451F_0B2F',
 'sensor': 'C-SAR',
 'startTime': '2016-09-02T03:34:20.000000',
 'stopTime': '2016-09-02T03:34:47.000000',
 'temporalBaseline': 516,
 'url': 'htt